In [1]:
import numpy as np
import pandas as pd
import multiprocessing

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 800)

import scipy

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.tsatools import detrend

import datetime as dt

Loading BokehJS ...

# Data

In [2]:
dat0 = pd.read_csv('data/ch4k_df.csv')
ref_dat0 = pd.read_csv('data/Article reference data.csv', low_memory = False, error_bad_lines = False, 
                       usecols = ['article_no', 'model_no', 'art_desc', 'sports_cat_desc', 'rmh_cat_desc', 
                                  'franchise', 'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc',
                                  'brand_desc', 'bus_unit_desc', 'rmh_cat_desc'])

# Remove clearance transactions!!
dat0['clearance'] = dat0.clearance.fillna(0) 
dat0['net_qty'] = (1 - dat0.clearance)*dat0.net_qty

# Magic

In [3]:
dat = dat0.copy()

# dat = dat[(dat.net_qty > 0) & (dat.season.isin(['FW15', 'FW16', 'FW17', 'FW18', 'FW19']))].round()

dat = dat[(dat.net_qty > 0) & (dat.season.isin(['SS15', 'SS16', 'SS17', 'SS18', 'SS19']))].round()

In [4]:

SS19 = pd.read_excel('data/ecom_SS19.xlsx').dropna()
SS19.shape

SS19 = SS19[SS19.carryover_FW18 == 'NO'] # new articles only


(7604, 3)

In [5]:
dat = dat[['article_number', 'season', 'season_net_qty', 'art_desc', 'sports_cat_desc',
           'rmh_cat_desc', 'franchise', 'gender_desc', 'age_group_desc',
           'prod_grp_desc', 'prod_type_desc', 'price', 'margin', 'cost', 'buy_availability']].drop_duplicates()

dat = dat[dat.season_net_qty > 200]
dat[['price', 'margin', 'cost']] = dat[['price', 'margin', 'cost']] # .fillna(0).astype('int')

dat = (
    dat.
    sort_values(['article_number', 'season']).
    drop_duplicates(subset = 'article_number').
    set_index('article_number').
    dropna()
      )


In [10]:
# art = np.random.choice(dat.index, size = 500, replace = False)

# Just articles new in SS19, with season_net_qty > 100
articles = set(SS19.article_number).intersection(set(dat.index))
art = articles.copy()

In [7]:
# art = np.random.choice(list(articles), size = 10, replace = False)
len(articles)
a

3460

In [12]:
from itertools import permutations 
from itertools import combinations

d = {}
cats = ['sports_cat_desc', 'rmh_cat_desc', 'franchise', 'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc']

# Create tidy dataframe, add results to it

for a in art:
    p = dat.loc[a, 'price']
    
    net_qtys = pd.Series()
    
    dat_p = dat[(dat.price >= 0.9*p) & (dat.price <= 1.1*p)]
    
    for c in cats: 
        dat_a = dat_p[dat_p[c] == dat_p.loc[a, c]] # filter to that level of that category        
        net_qtys = net_qtys.append(dat_a.season_net_qty)
    
    for c2 in combinations(cats, 2):
        dat_a = dat_p[(dat_p[c2[0]] == dat_p.loc[a, c2[0]]) &
                      (dat_p[c2[1]] == dat_p.loc[a, c2[1]])]
        net_qtys = net_qtys.append(dat_a.season_net_qty)
        
    for c3 in combinations(cats, 3):
        dat_a = dat_p[(dat_p[c3[0]] == dat_p.loc[a, c3[0]]) &
                      (dat_p[c3[1]] == dat_p.loc[a, c3[1]]) &
                      (dat_p[c3[2]] == dat_p.loc[a, c3[2]])]
        net_qtys = net_qtys.append(dat_a.season_net_qty)
    
    for c4 in combinations(cats, 4):
        dat_a = dat_p[(dat_p[c4[0]] == dat_p.loc[a, c4[0]]) &
                      (dat_p[c4[1]] == dat_p.loc[a, c4[1]]) &
                      (dat_p[c4[2]] == dat_p.loc[a, c4[2]]) &
                      (dat_p[c4[3]] == dat_p.loc[a, c4[3]])]
        net_qtys = net_qtys.append(dat_a.season_net_qty)
    
    for c4 in combinations(cats, 5):
        dat_a = dat_p[(dat_p[c4[0]] == dat_p.loc[a, c4[0]]) &
                      (dat_p[c4[1]] == dat_p.loc[a, c4[1]]) &
                      (dat_p[c4[2]] == dat_p.loc[a, c4[2]]) &
                      (dat_p[c4[3]] == dat_p.loc[a, c4[3]]) &
                      (dat_p[c4[4]] == dat_p.loc[a, c4[4]])]
        net_qtys = net_qtys.append(dat_a.season_net_qty)
    
    for c4 in combinations(cats, 6):
        dat_a = dat_p[(dat_p[c4[0]] == dat_p.loc[a, c4[0]]) &
                      (dat_p[c4[1]] == dat_p.loc[a, c4[1]]) &
                      (dat_p[c4[2]] == dat_p.loc[a, c4[2]]) &
                      (dat_p[c4[3]] == dat_p.loc[a, c4[3]]) &
                      (dat_p[c4[4]] == dat_p.loc[a, c4[4]]) &
                      (dat_p[c4[5]] == dat_p.loc[a, c4[5]])]
        net_qtys = net_qtys.append(dat_a.season_net_qty)
    
    d[a] = {
        'mean': net_qtys.mean(),
        'max': net_qtys.max(),
        '50': np.percentile(net_qtys, 50),
        '70': np.percentile(net_qtys, 70),
        '80': np.percentile(net_qtys, 80),
        '90': np.percentile(net_qtys, 90),
        'length': len(net_qtys)
           }
    if len(d) % 50 == 0:
        print(len(d))
        
        

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450


In [ ]:
d.keys()
d['F34164']
d['F34164']['mean']


In [13]:
dat_art = dat[dat.index.isin(art)]


In [28]:
preds = pd.DataFrame([(a, d[a]['mean'], d[a]['70'], d[a]['length']) for a in d.keys()]).round()
preds.columns = ('article_number', 'mean', 'pred_70', 'length')
preds = preds.set_index('article_number')

preds.head()

,mean,pred_70,length
article_number,,,
D98181,684.0,690.0,6648
F35787,693.0,677.0,21779
DB3361,957.0,976.0,20822
BD7160,774.0,514.0,1165
DU2042,681.0,565.0,11941


In [33]:
dat_art2 = pd.merge(
    dat_art, 
    preds,
    left_index=True, right_index=True,
    how = 'left'
)

dat_art2 = dat_art2[['season', 'season_net_qty', 
                     'mean', 'pred_70',
                     'art_desc', 'sports_cat_desc',
       'rmh_cat_desc', 'franchise', 'gender_desc', 'age_group_desc',
       'prod_grp_desc', 'prod_type_desc', 'price', 'margin', 'cost']] # .sort_values('APE', ascending = False)

In [43]:
APE = (abs(dat_art2.season_net_qty - dat_art2.pred_70)/dat_art2.season_net_qty*100).round()
APE.mean()

77.91040462427746

In [ ]:
# Criteria:
# (1) in transaction data
# (2) season_net_qty > 200 
# (3) in eCom SS19 range file as non-carryover 


In [36]:
dat_art4 = pd.merge(
    dat_art2, 
    SS19[['article_number', 'Ecom_FC_RMA']],
    left_index = True, right_on = 'article_number', how = 'left'
).drop('article_number', axis = 1)

In [20]:
# Profit
def P(d, margin, cost, b):
    if d > b:    # CANNOT satisfy demand
        return b*margin
    
    elif d <= b: # CAN satisfy demand
        return d*margin - (b - d)*cost
    
    else:
        print('Error')

In [37]:
#P(row['season_net_qty'], row['margin'], row['cost'], row['pred'])

dat_art4['eCom_profit'] = dat_art4.apply(lambda row: P(row['season_net_qty'], row['margin'], row['cost'], row['Ecom_FC_RMA']), axis=1)
dat_art4['DAA_profit_m']  = dat_art4.apply(lambda row: P(row['season_net_qty'], row['margin'], row['cost'], row['mean']), axis=1)
dat_art4['DAA_profit_70']  = dat_art4.apply(lambda row: P(row['season_net_qty'], row['margin'], row['cost'], row['pred_70']), axis=1)


# for pct in ['pred', 'pred_50', 'pred_60', 'pred_70', 'pred_75', 'pred_80', 'pred_90', 'pred_98']:
#     col = 'DAA_profit' + '_' + pct
#     dat_art4[col]  = dat_art4.apply(lambda row: P(row['season_net_qty'], row['margin'], row['cost'], row[pct]), axis=1)
    
                          

In [46]:
dat_art4[['DAA_profit_70', 'eCom_profit']].sum()

(73 - 58)/73

DAA_profit_70    58465728.0
eCom_profit      73359849.0
dtype: float64

0.2054794520547945

In [ ]:
# Histogram

plt.hist(net_qtys, bins = [0, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 5000], density = True)

pass;

In [ ]:
# Empirical distributions

import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF

ecdf = ECDF(net_qtys)

ecdf([100, 500, 1000, 3000])

np.percentile(net_qtys, 75)
net_qtys.describe()

In [ ]:
# ---- Plot -----
# a = np.random.choice(preds.article_number.unique(), size = 1, replace = False)[0]
aoi = 'G26535'
a = aoi

dat_a = preds[preds.article_number == a][['week', 'net_qty', 'corrected', 'y_hat']]
        

plt.rcParams["figure.figsize"] = [10,7]
dat_a.sort_values('week').set_index('week').plot(linewidth = 3)
dat_a.sort_values('week').set_index('week').round()

dat_a[['net_qty', 'corrected', 'y_hat']].apply(np.sum).round()

preds_season.reset_index()[preds_season.index == a]

dat_aoi = dat0[dat0.article_number == aoi].copy()

dat_aoi = pd.merge(
    pd.DataFrame(dat_aoi.groupby(['year', 'week'])['net_qty'].sum()).reset_index(),
    dat_aoi[['year', 'week']].drop_duplicates()
)

dat_aoi.year = [str(x) for x in dat_aoi.year]
dat_aoi.week = [str(x) for x in dat_aoi.week]
dat_aoi['date'] = [dt.datetime.strptime(x[0] + '-' + x[1] + '-1', "%Y-%W-%w") for x in zip(dat_aoi.year, dat_aoi.week)]

plt.rcParams["figure.figsize"] = [10,7]
dat_aoi[['date', 'net_qty']].set_index('date').plot(linewidth = 4)

# dat_aoi

# Optimal Buy

In [ ]:
dat[dat.index == art[0]][['season', 'season_net_qty', 'sports_cat_desc', 'rmh_cat_desc', 'franchise', 'price', 'margin', 'cost']]

minimize_EL(net_qtys, dat.loc[a, 'margin'], dat.loc[a, 'cost'])
net_qtys.describe()


In [ ]:
from functools import partial
from scipy import optimize
from scipy import integrate
import scipy.stats as stats

In [ ]:
# Loss --- demand, buy, margin, cost
def L(d, b, margin, cost):
    if d > b:    # CANNOT satisfy demand
        return (d - b)*margin
    
    elif d <= b: # CAN satisfy demand
        return (b - d)*cost
    
    else:
        print('Error')

# E[L | buy, article_mean, article_sd, article_margin, article_cost]
def EL(net_qtys, margin, cost, b):
    return sum([L(x, b, margin, cost) for x in net_qtys])/len(net_qtys)

def minimize_EL(net_qtys, margin, cost):
    p = partial(EL, net_qtys, margin, cost) # Make EL function of only one var
    mu = np.mean(net_qtys)
    buy_opt = optimize.minimize_scalar(p, bounds = (mu, mu + 2*np.std(net_qtys)))
    return int(buy_opt['x']) # optimal buy quantity

In [ ]:
# Profit
def P(d, margin, cost, b):
    if d > b:    # CANNOT satisfy demand
        return b*margin
    
    elif d <= b: # CAN satisfy demand
        return d*margin - (b - d)*cost
    
    else:
        print('Error')

def EP(net_qtys, margin, cost, b):
    return -sum([P(x, margin, cost, b) for x in net_qtys])Yeah /len(net_qtys) # integral--by rectangle method, each w/ 1/n height

def maximize_EP(net_qtys, margin, cost):
    p = partial(EP, net_qtys, margin, cost) # Make EL function of only one var
    mu = np.mean(net_qtys)
    buy_opt = optimize.minimize_scalar(p, bounds = (mu, mu + 2*np.std(net_qtys)))
    return int(buy_opt['x']) # optimal buy quantity

In [ ]:
# one offs

# # Minimize Loss
# l = partial(EL, net_qtys, 63, 30) # Make EL function of only one var
# [print(x, ':', round(l(x))) for x in range(0, 600, 50)]

# # Maximize Profit
# p = partial(EP, net_qtys, 63, 30)
# [print(x, ':', round(p(x))) for x in range(0, 600, 50)]


minimize_EL(net_qtys, 6, 2)
                          
maximize_EP(net_qtys, 6, 2)

# HUZZAH!!!!!


In [ ]:


# --- Overbuy statistics ---
# preds_season['pct_overbuy'] = (preds_season.Opt_Ovb - preds_season.y_hat)/preds_season.y_hat*100
# b = np.array([0, 1000, 2000, 5000, 10000, 50000])
# preds_season['bins'] = pd.cut(preds_season.y_hat, bins = b)
# preds_season.groupby('bins')['pct_overbuy'].describe().round()


# Appendix

In [ ]:
dat_art5[dat_art5.pred_90 < dat_art5.season_net_qty].sum()

# season_net_qty         227889.0
# pred_80                129573.0
# Ecom_FC_RMA            252412.0
# DAA_profit_pred_80    4725638.0
# eCom_profit           6433454.0
# diff                  2266284.0
# dtype: float64

# Comments: 
    # when we under-forecast... we DRAMATICALLY under-forecast
    # HUGE difference in profit

dat_art5[dat_art5.pred_90 >= dat_art5.season_net_qty].sum()

# season_net_qty         179877.0
# pred_80                354269.0
# Ecom_FC_RMA            310015.0
# DAA_profit_pred_80    4232213.0
# eCom_profit           4550809.0
# diff                  2017816.0
# dtype: float64

# Comments: 
    # in sum, we just slightly over-forecast
    # Tiny difference in profit
    